In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from funpymodeling.exploratory import freq_tbl 
import re

In [ ]:
dfp=pd.read_csv("listingsPorto.csv")
dfa=pd.read_csv("listingsAmsterdam.csv")

***AMSTERDAM***

In [ ]:
dfa.head(5)
dfa.dtypes

In [ ]:
#Se eliminan los valores nulos
dfa.isnull().sum()
dfa=dfa.fillna(method='ffill')
dfa=dfa.fillna(method='bfill')

In [ ]:
#Como no funciono el bfill y ffill usamos el metodo de sustituir por una palabra 
dfa1=dfa.fillna('nulos')
dfa1.info()
dfa1.isnull().sum().sum()

In [ ]:
#Reemplazamos el $ en la columna price para que se vuelva numerica 
dfa1['price'] = dfa1['price'].str.replace('$', '', regex=False)
dfa1['price'] = dfa1['price'].str.replace(',', '', regex=False)
dfa1['price'] = pd.to_numeric(dfa1['price'])

In [ ]:
dfa1['price']

In [ ]:
#Se separan las columnas por cualitativas y cuantitativas
numcol=dfa1.select_dtypes(include=['number'])
catcolsinfiltrar=dfa1.select_dtypes(include=['object'])

In [ ]:
#Se seleccionan las variables solicitadas 
catcol=catcolsinfiltrar[['host_response_time','host_acceptance_rate','property_type','room_type','amenities','host_is_superhost']]

In [ ]:
#Se procesan los datos nulos de las varibales numericas 
numcol.isnull().sum().sum()
numcol=numcol.fillna(method='ffill')
numcol=numcol.fillna(method='bfill')

In [ ]:
#Comprobamos los valores faltantes nuevamente 
numcol.isnull().sum().sum()

In [ ]:
numcol_sinid=numcol.drop(['id','host_id','scrape_id'],axis=1)

In [ ]:
#Obtenemos los limites superior e inferior de todas las variables numericas
#Realizamos diagrama de caja o bigote de cada columna del dataframe
fig = plt.figure(figsize =(15, 8))
numcol_sinid.plot(kind='box', vert=False)
plt.title("Valores Atípicos del Dataframe")
plt.show() 

In [ ]:
#Método aplicando desviación estandar. Encuentro los valores extremos
y=numcol_sinid
Limite_Superior= y.mean() + 3*y.std()
Limite_Inferior= y.mean() - 3*y.std()
print("Limite superior permitido", Limite_Superior)
print("Limite inferior permitido", Limite_Inferior)

In [ ]:
#Obtenemos datos y los outliers se convierten en nulos en el DataFrame
numcol_outliers= numcol_sinid[(y<=Limite_Superior)&(y>=Limite_Inferior)]
numcol_outliers

In [ ]:
#Comprobamos los valores atipicos que se volvieron nulos 
numcol_outliers.isnull().sum()

In [ ]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
numcol_clean=numcol_outliers.copy()
numcol_clean=numcol_clean.fillna(round(numcol_outliers.mean(),1))
numcol_clean

In [ ]:
#Corroboramos valores nulos del dataframe LIMPIO
numcol_clean.isnull().sum().sum()

In [ ]:
freq_tbl(catcol)
catcol.columns

In [ ]:
# Limitar numero máximo de filas
pd.options.display.max_rows = None

In [ ]:
#Obtenemos un análisis univariado de las diferentes variables
table1= freq_tbl(catcol['amenities'])
table2= freq_tbl(catcol['host_acceptance_rate']) #ind ind
table3= freq_tbl(catcol['host_is_superhost'])  
table4= freq_tbl(catcol['host_response_time']) #dep ind
table5= freq_tbl(catcol['property_type'])
table6= freq_tbl(catcol['room_type'])
table1
table2
table3
table4
table5
table6

In [ ]:
table1.amenities = table1.amenities.replace({'["Smoke alarm", "Washer", "Wifi", "TV", "Kitchen"]':'Amenidades 1'}, regex=False)
table1.amenities = table1.amenities.replace({'["Iron", "Paid parking on premises", "Air conditioning", "Smoke alarm", "Dedicated workspace", "Luggage dropoff allowed", "Wifi", "Hot water", "Essentials", "Heating", "Shampoo", "TV"]':'Amenidades 2'}, regex=False)
table1.amenities = table1.amenities.replace({r'["Iron", "Shampoo", "Hot water kettle", "Exterior security cameras on property", "Paid parking off premises", "Extra pillows and blankets", "Shower gel", "Paid washer \u2013 In building", "Body soap", "Paid dryer \u2013 In building", "Hangers", "Dedicated workspace", "First aid kit", "Host greets you", "Bed linens", "TV", "Conditioner", "Breakfast", "Mini fridge", "Luggage dropoff allowed", "Safe", "Waterfront", "Portable fans", "Hair dryer", "Smoke alarm", "Clothing storage", "Refrigerator", "Shared patio or balcony", "Wifi", "Hot water", "Coffee maker", "Essentials", "Heating", "Fire extinguisher"]':'Amenidades 3'}, regex=False)
table1.amenities = table1.amenities.replace({'["Iron", "Shampoo", "Paid parking on premises", "Extra pillows and blankets", "Stove", "Hangers", "Dedicated workspace", "Carbon monoxide alarm", "First aid kit", "Self check-in", "Bed linens", "TV", "Microwave", "Washer", "Luggage dropoff allowed", "Oven", "Cooking basics", "Dishwasher", "Air conditioning", "Smoke alarm", "Dryer", "Wifi", "Hot water", "Coffee maker", "Essentials", "Heating", "Fire extinguisher", "Building staff", "Gym", "Dishes and silverware", "Kitchen"]':'Amenidades 4'}, regex=False)


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro= table1[table1['frequency']>7]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index1= Filtro.set_index('amenities')
Filtro_index1
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency1= Filtro_index1.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency1

In [ ]:
Filtro_index_frequency1.plot(kind = 'bar', width=0.8, figsize=(8,4),color='orchid')
plt.title('Herramientas de relación con los clientes')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')

In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada porcentaje de aceptacion de arrendador

Filtro= table2[table2['frequency']>281]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index2= Filtro.set_index('host_acceptance_rate')
Filtro_index2
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency2= Filtro_index2.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency2

In [ ]:
#Realizamos grafico de pastel del dataframe filtrado
Filtro_index2["frequency"].plot(kind='pie', figsize=(10,5), shadow=True, autopct="%0.1f%%")


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro= table3[table3['frequency']>7]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index3= Filtro.set_index('host_is_superhost')
Filtro_index3
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency3= Filtro_index3.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency3

In [ ]:
#Realizamos grafico de barras del dataframe filtrado
Filtro_index_frequency3.plot(kind = 'bar', width=0.8, figsize=(8,4),color='red')
plt.title('Anfitrión es Super Anfitrión')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')

In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro= table4[table4['frequency']>1]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index4= Filtro.set_index('host_response_time')
Filtro_index4
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency4= Filtro_index4.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency4

In [ ]:

#Realizamos grafico de área del dataframe filtrado
Filtro_index_frequency4.plot(kind='area', figsize=(10,4),alpha = 0.5,color='green')

In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro= table5[table5['frequency']>100]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index5= Filtro.set_index('property_type')
Filtro_index5
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency5= Filtro_index5.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency5

In [ ]:
#Realizamos grafico de barras del dataframe filtrado
Filtro_index_frequency5.plot(kind = 'bar', width=0.8, figsize=(8,4),color='powderblue')
plt.title('Tipo de Propiedad')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro= table6[table6['frequency']>1]
Filtro
#Ajustamos el indice de mi dataframe
Filtro_index6= Filtro.set_index('room_type')
Filtro_index6
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency6= Filtro_index6.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency6

In [ ]:
#Realizamos grafico de área del dataframe filtrado
Filtro_index_frequency6.plot(kind='area', figsize=(10,4),alpha = 0.5,color="orange")

In [ ]:
# Unimos los otros data frame para ver que esten limpios

dfa_limpio = pd.concat([numcol_clean,numcol,catcol], axis=1)
dfa_limpio.head(3)

In [ ]:
dfa_limpio.isnull().sum().sum()

***Procedemos a trabajar con Porto***

In [ ]:
dfp.head(5)
dfp.dtypes

In [ ]:
#Se eliminan los valores nulos
dfp.isnull().sum()
dfp=dfp.fillna(method='ffill')
dfp=dfp.fillna(method='bfill')

In [ ]:
#Como no funciono el bfill y ffill usamos el metodo de sustituir por una palabra 
dfp1=dfp.fillna('nulos')
dfp1.info()
dfp1.isnull().sum().sum()

In [ ]:
#Reemplazamos el $ en la columna price para que se vuelva numerica 
dfp1['price'] = dfp1['price'].str.replace('$', '', regex=False)
dfp1['price'] = dfp1['price'].str.replace(',', '', regex=False)
dfp1['price'] = pd.to_numeric(dfa1['price'])

In [ ]:
dfp1['price']

In [ ]:
#Se separan las columnas por cualitativas y cuantitativas
numcolp=dfp1.select_dtypes(include=['number'])
catcolsinfiltrarp=dfp1.select_dtypes(include=['object'])

In [ ]:
#Se seleccionan las variables solicitadas 
catcolp=catcolsinfiltrarp[['host_response_time','host_acceptance_rate','property_type','room_type','amenities','host_is_superhost']]

In [ ]:
#Se procesan los datos nulos de las varibales numericas 
numcolp.isnull().sum().sum()
numcolp=numcolp.fillna(method='ffill')
numcolp=numcolp.fillna(method='bfill')

In [ ]:
#Comprobamos los valores faltantes nuevamente 
numcolp.isnull().sum().sum()

In [ ]:
numcolp_sinid=numcolp.drop(['id','host_id','scrape_id'],axis=1)

In [ ]:
#Obtenemos los limites superior e inferior de todas las variables numericas
#Realizamos diagrama de caja o bigote de cada columna del dataframe
fig = plt.figure(figsize =(15, 8))
numcolp_sinid.plot(kind='box', vert=False)
plt.title("Valores Atípicos del Dataframe")
plt.show() 

In [ ]:
#Método aplicando desviación estandar. Encuentro los valores extremos
y=numcolp_sinid
Limite_Superiorp= y.mean() + 3*y.std()
Limite_Inferiorp= y.mean() - 3*y.std()
print("Limite superior permitido", Limite_Superior)
print("Limite inferior permitido", Limite_Inferior)

In [ ]:
#Obtenemos datos y los outliers se convierten en nulos en el DataFrame
pnumcol_outliers= numcolp_sinid[(y<=Limite_Superiorp)&(y>=Limite_Inferiorp)]
pnumcol_outliers

In [ ]:
#Comprobamos los valores atipicos que se volvieron nulos 
pnumcol_outliers.isnull().sum()

In [ ]:
#Reemplazamos valores atípicos (nulos) del dataframe con "mean"
#Realizamos una copia del dataframe
pnumcol_clean=pnumcol_outliers.copy()
pnumcol_clean=pnumcol_clean.fillna(round(pnumcol_outliers.mean(),1))
pnumcol_clean

In [ ]:
#Corroboramos valores nulos del dataframe LIMPIO
pnumcol_clean.isnull().sum().sum()

In [ ]:
freq_tbl(catcolp)
catcolp.columns

In [ ]:
# Limitar numero máximo de filas
pd.options.display.max_rows = None

In [ ]:
#Obtenemos un análisis univariado de las diferentes variables
table1p= freq_tbl(catcolp['amenities'])
table2p= freq_tbl(catcolp['host_acceptance_rate'])
table3p= freq_tbl(catcolp['host_is_superhost'])
table4p= freq_tbl(catcolp['host_response_time'])
table5p= freq_tbl(catcolp['property_type'])
table6p= freq_tbl(catcolp['room_type'])
table1p
table2p
table3p
table4p
table5p
table6p

In [ ]:
table1p.amenities = table1p.amenities.replace({'["Hair dryer", "Kitchen", "Free parking on premises", "Outdoor dining area", "Wifi", "Dedicated workspace", "Washer", "Smoke alarm", "BBQ grill", "TV", "Heating", "Fire extinguisher", "Cooking basics", "First aid kit", "Carbon monoxide alarm"]':'Amenidades 2'}, regex=False)
table1p.amenities = table1p.amenities.replace({'["Hair dryer", "Essentials", "Kitchen", "Outdoor dining area", "Wifi", "Dedicated workspace", "Smoke alarm", "Iron", "TV", "Exterior security cameras on property", "Hangers", "Air conditioning", "Heating", "Fire extinguisher", "First aid kit", "Long term stays allowed", "Carbon monoxide alarm", "Hot water", "Luggage dropoff allowed"]':'Amenidades 2'}, regex=False)
table1p.amenities = table1p.amenities.replace({'["Lockbox", "Fire extinguisher", "First aid kit", "Refrigerator", "Dishes and silverware", "Hot water kettle", "TV", "Stove", "Kitchen", "Essentials", "Microwave", "Shampoo", "Cooking basics", "Coffee maker", "Wifi", "Hair dryer", "Hangers", "Air conditioning", "Long term stays allowed", "Hot water", "Self check-in"]':'Amenidades 3'}, regex=False)


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtrop= table1p[table1p['frequency']>17]
Filtrop
#Ajustamos el indice de mi dataframe
Filtro_index1p= Filtrop.set_index('amenities')
Filtro_index1p
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency1p= Filtro_index1p.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency1p

In [ ]:
Filtro_index_frequency1p.plot(kind = 'bar', width=0.8, figsize=(8,4),color='orchid')
plt.title('Herramientas de relación con los clientes')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')

In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada porcentaje de aceptacion de arrendador

Filtro10= table2p[table2p['frequency']>281]
Filtro10
#Ajustamos el indice de mi dataframe
Filtro_index20= Filtro10.set_index('host_acceptance_rate')
Filtro_index20
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency20= Filtro_index20.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency20

In [ ]:
#Realizamos grafico de pastel del dataframe filtrado
Filtro_index20["frequency"].plot(kind='pie', figsize=(10,5), shadow=True, autopct="%0.1f%%")


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro20= table3p[table3p['frequency']>7]
Filtro20
#Ajustamos el indice de mi dataframe
Filtro_index30= Filtro20.set_index('host_is_superhost')
Filtro_index30
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency30= Filtro_index30.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency30

In [ ]:
#Realizamos grafico de barras del dataframe filtrado
Filtro_index_frequency30.plot(kind = 'bar', width=0.8, figsize=(8,4),color='red')
plt.title('Anfitrión es Super Anfitrión')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')

In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro03= table4p[table4p['frequency']>1]
Filtro03
#Ajustamos el indice de mi dataframe
Filtro_index40= Filtro03.set_index('host_response_time')
Filtro_index40
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency40= Filtro_index40.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency40

In [ ]:

#Realizamos grafico de área del dataframe filtrado
Filtro_index_frequency40.plot(kind='area', figsize=(10,4),alpha = 0.5,color='green')


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro04= table5p[table5p['frequency']>100]
Filtro04
#Ajustamos el indice de mi dataframe
Filtro_index50= Filtro04.set_index('property_type')
Filtro_index50
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency50= Filtro_index50.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency50

In [ ]:
#Realizamos grafico de barras del dataframe filtrado
Filtro_index_frequency50.plot(kind = 'bar', width=0.8, figsize=(8,4),color='powderblue')
plt.title('Tipo de Propiedad')
plt.xlabel('Tipo de Herramienta')
plt.ylabel('Frecuencia')


In [ ]:
#Creamos un filtro de los valores más reelevantes de la variables categórica seleccionada

Filtro05= table6p[table6p['frequency']>1]
Filtro05
#Ajustamos el indice de mi dataframe
Filtro_index60= Filtro05.set_index('room_type')
Filtro_index60
#Eliminamos las columnas de "percentage" y "cumulative_perc"
Filtro_index_frequency60= Filtro_index60.drop(['percentage','cumulative_perc'], axis=1)
Filtro_index_frequency60

In [ ]:
#Realizamos grafico de área del dataframe filtrado
Filtro_index_frequency60.plot(kind='area', figsize=(10,4),alpha = 0.5,color="orange")

In [ ]:
# Unimos los otros data frame para ver que esten limpios
dfp_limpio = pd.concat([pnumcol_clean,numcolp,catcolp], axis=1)
dfp_limpio.head(3)

In [ ]:
dfp_limpio.isnull().sum().sum()

**Análisis de Regresión**

Amsterdam

In [ ]:
#Imprimimos el scatter plot entre la variable dependiente e independiente
#para observar el comportamiento en su dispersión 
sns.scatterplot(x='bedrooms', y='price', color="blue", data=numcol_clean)


In [ ]:
sns.scatterplot(x='review_scores_rating', y='price', color="blue", data=numcol_clean)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep= numcol_clean[['bedrooms']]
Var_Dep= numcol_clean['price']

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep2= numcol_clean[['maximum_nights']]
Var_Dep2= numcol_clean['price']

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep3= numcol_clean[['review_scores_rating']]
Var_Dep3= numcol_clean['price']

In [ ]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
model= LinearRegression()
model2 = LinearRegression()
model3 = LinearRegression()

In [ ]:
#Verificamos la función relacionada al modelo
type(model)
type(model2)
type(model3)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model.fit(X=Vars_Indep, y=Var_Dep)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model2.fit(X=Vars_Indep2, y=Var_Dep2)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model3.fit(X=Vars_Indep3, y=Var_Dep3)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
model.__dict__
model2.__dict__
model3.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
model.score(Vars_Indep,Var_Dep)

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
model2.score(Vars_Indep2,Var_Dep2)

In [ ]:
model3.score(Vars_Indep3,Var_Dep3)

In [ ]:
#Predecimos los valores de total de accidentes a partir de las variables
y_pred= model.predict(X=numcol_clean[['bedrooms']])
y_pred

y_pred2=model2.predict(X=numcol_clean[['maximum_nights']])
y_pred2

y_pred3=model3.predict(X=numcol_clean[['review_scores_rating']])
y_pred3

In [ ]:
#Insertamos la columna de predicciones en el DataFrame
numcol_clean.insert(0, 'Predicciones', y_pred)
#numcol_clean

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x='bedrooms', y='price', color="blue", data=numcol_clean)
sns.scatterplot(x='bedrooms', y='Predicciones', color="red", data=numcol_clean)

In [ ]:
sns.scatterplot(x='maximum_nights', y='price', color="blue", data=numcol_clean)
sns.scatterplot(x='maximum_nights', y='Predicciones', color="red", data=numcol_clean)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x='review_scores_rating', y='price', color="blue", data=numcol_clean)
sns.scatterplot(x='review_scores_rating', y='Predicciones', color="red", data=numcol_clean)

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Deter=model.score(X=Vars_Indep, y=Var_Dep)
coef_Deter
coef_Deter2=model2.score(X=Vars_Indep2, y=Var_Dep2)
coef_Deter2
coef_Deter3=model3.score(X=Vars_Indep3, y=Var_Dep3)
coef_Deter3

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correl=np.sqrt(coef_Deter)
coef_Correl
coef_Correl2=np.sqrt(coef_Deter2)
coef_Correl2
coef_Correl3=np.sqrt(coef_Deter3)
coef_Correl3

In [ ]:
#Graficamos todas las dispersiones entre todas las variables
#sns.pairplot(df)
#Saltamos esta funcion por el tiempo que tarda en ejecutarse 

In [ ]:
#Encontramos todas las correlaciones entre las variables 
Corr_Factors=numcol_clean.corr()
Corr_Factors

In [ ]:
#Encontramos el valor absoluto de todas las correlaciones entre las variables 
Corr_Factors1= abs(Corr_Factors)
Corr_Factors1

In [ ]:
#Graficamos el mapa de calor de los coeficientes de correlación 
Heat_Map= sns.heatmap(Corr_Factors1, cmap = 'coolwarm')  
Heat_Map

In [ ]:
#Ajustamos el mapa de calor de los coeficientes de correlación 
Heat_Map= sns.heatmap(Corr_Factors1, cmap = 'Oranges', annot=True, fmt=".2f")  
Heat_Map

In [ ]:
numcol_clean.dtypes

In [ ]:
# Graficar múltiples scatterplots en una sola gráfica
sns.scatterplot(x='bedrooms', y='price', color="blue", label='Bedrooms', data=numcol_clean)
sns.scatterplot(x='maximum_nights', y='price', color="red", label='Maximum Nights', data=numcol_clean)
sns.scatterplot(x='review_scores_rating', y='price', color="yellow", label='Review Scores', data=numcol_clean)
sns.scatterplot(x='bathrooms', y='price', color="orchid", label='Bathrooms', data=numcol_clean)
sns.scatterplot(x='accommodates', y='price', color="gray", label='Accommodates', data=numcol_clean)

# Mostrar la leyenda para identificar las series
plt.legend()

In [ ]:
# Graficar múltiples scatterplots en una sola gráfica
sns.scatterplot(x='bedrooms', y='price', color="blue", label='Bedrooms', data=numcol_clean)
plt.show()
sns.scatterplot(x='maximum_nights', y='price', color="red", label='Maximum Nights', data=numcol_clean)
plt.show()
sns.scatterplot(x='review_scores_rating', y='price', color="yellow", label='Review Scores', data=numcol_clean)
plt.show()
sns.scatterplot(x='bathrooms', y='price', color="orchid", label='Bathrooms', data=numcol_clean)
plt.show()
sns.scatterplot(x='accommodates', y='price', color="gray", label='Accommodates', data=numcol_clean)
plt.show()

In [ ]:
Vars_Indepm= numcol_clean[['bedrooms','maximum_nights','review_scores_rating','bathrooms','accommodates']]
Var_Depm= numcol_clean['price']

In [ ]:
modelm= LinearRegression()

In [ ]:
#Verificamos la función relacionada al modelo
type(modelm)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
modelm.fit(X=Vars_Indepm, y=Var_Depm)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
modelm.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
modelm.score(Vars_Indepm,Var_Depm)

In [ ]:
y_predm= modelm.predict(X=numcol_clean[['bedrooms','maximum_nights','review_scores_rating','bathrooms','accommodates']])
y_predm

In [ ]:
numcol_clean.insert(0, 'Predicciones', y_predm)

In [ ]:
# Crear un gráfico de dispersión para cada variable predictora
for column in ['bedrooms', 'maximum_nights', 'review_scores_rating', 'bathrooms', 'accommodates']:
    plt.figure(figsize=(8, 6))
    
    # Gráfico de los valores reales
    sns.scatterplot(x=column, y='price', data=numcol_clean, color="blue", label='Precio Real')
    
    # Gráfico de las predicciones
    sns.scatterplot(x=column, y='Predicciones', data=numcol_clean, color="red", label='Precio Predicho')
    
    # Añadir título y leyenda
    plt.title(f'Comparación entre {column} y Precio')
    plt.legend()
    
    # Mostrar la gráfica
    plt.show()

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Determ=modelm.score(X=Vars_Indepm, y=Var_Depm)
coef_Determ

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correlm=np.sqrt(coef_Determ)
coef_Correlm

In [ ]:
# Unimos los otros data frame para ver que esten limpios

dfa_limpio = pd.concat([numcol_clean,numcol,catcol], axis=1)
dfa_limpio.head(3)

**Porto**

In [ ]:
#Imprimimos el scatter plot entre la variable dependiente e independiente
#para observar el comportamiento en su dispersión 
sns.scatterplot(x='bedrooms', y='price', color="blue", data=pnumcol_clean)


In [ ]:
sns.scatterplot(x='maximum_nights', y='price', color="blue", data=pnumcol_clean)

In [ ]:
sns.scatterplot(x='review_scores_rating', y='price', color="blue", data=pnumcol_clean)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indepp= pnumcol_clean[['bedrooms']]
Var_Depp= pnumcol_clean['price']

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep2p =pnumcol_clean[['maximum_nights']]
Var_Dep2p= pnumcol_clean['price']

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep3p= pnumcol_clean[['review_scores_rating']]
Var_Dep3p= pnumcol_clean['price']

In [ ]:
#Se define model como la función de regresión lineal
from sklearn.linear_model import LinearRegression
modelp= LinearRegression()
model2p = LinearRegression()
model3p = LinearRegression()

In [ ]:
#Verificamos la función relacionada al modelo
type(modelp)
type(model2p)
type(model3p)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
modelp.fit(X=Vars_Indepp, y=Var_Depp)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model2p.fit(X=Vars_Indep2p, y=Var_Dep2p)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
model3p.fit(X=Vars_Indep3p, y=Var_Dep3p)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
modelp.__dict__
model2p.__dict__
model3p.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
modelp.score(Vars_Indepp,Var_Depp)

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
model2p.score(Vars_Indep2p,Var_Dep2p)

In [ ]:
model3p.score(Vars_Indep3p,Var_Dep3p)

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión lineal
Vars_Indep3p= numcol_clean[['review_scores_rating']]
Var_Dep3p= numcol_clean['price']

In [ ]:
#Predecimos los valores de total de accidentes a partir de las variables
y_predp= modelp.predict(X=pnumcol_clean[['bedrooms']])
y_predp

y_pred2p=model2p.predict(X=pnumcol_clean[['maximum_nights']])
y_pred2p

y_pred3p=model3.predict(X=pnumcol_clean[['review_scores_rating']])
y_pred3p

In [ ]:
#Insertamos la columna de predicciones en el DataFrame
pnumcol_clean.insert(0, 'Predicciones', y_predp)
#numcol_clean

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x='bedrooms', y='price', color="blue", data=pnumcol_clean)
sns.scatterplot(x='bedrooms', y='Predicciones', color="red", data=pnumcol_clean)

In [ ]:
sns.scatterplot(x='maximum_nights', y='price', color="blue", data=pnumcol_clean)
sns.scatterplot(x='maximum_nights', y='Predicciones', color="red", data=pnumcol_clean)

In [ ]:
#Visualizamos la gráfica comparativa entre el total real y el total predecido

sns.scatterplot(x='review_scores_rating', y='price', color="blue", data=pnumcol_clean)
sns.scatterplot(x='review_scores_rating', y='Predicciones', color="red", data=pnumcol_clean)

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Deterp=modelp.score(X=Vars_Indepp, y=Var_Depp)
coef_Deterp
coef_Deter2p=model2p.score(X=Vars_Indep2p, y=Var_Dep2p)
coef_Deter2p
coef_Deter3p=model3p.score(X=Vars_Indep3p, y=Var_Dep3p)
coef_Deter3p

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correlp=np.sqrt(coef_Deterp)
coef_Correlp
coef_Correl2p=np.sqrt(coef_Deter2p)
coef_Correl2p
coef_Correl3p=np.sqrt(coef_Deter3p)
coef_Correl3p

In [ ]:
#Graficamos todas las dispersiones entre todas las variables
#sns.pairplot(df)
#Saltamos esta funcion por el tiempo que tarda en ejecutarse 

In [ ]:
#Encontramos todas las correlaciones entre las variables 
Corr_Factorsp=pnumcol_clean.corr()
Corr_Factorsp

In [ ]:
#Encontramos el valor absoluto de todas las correlaciones entre las variables 
Corr_Factors1p= abs(Corr_Factorsp)
Corr_Factors1p

In [ ]:
#Graficamos el mapa de calor de los coeficientes de correlación 
Heat_Mapp= sns.heatmap(Corr_Factors1p, cmap = 'coolwarm')  
Heat_Mapp

In [ ]:
#Ajustamos el mapa de calor de los coeficientes de correlación 
Heat_Mapp= sns.heatmap(Corr_Factors1p, cmap = 'Oranges', annot=True, fmt=".2f")  
Heat_Mapp

In [ ]:
pnumcol_clean.dtypes

In [ ]:
# Graficar múltiples scatterplots en una sola gráfica
sns.scatterplot(x='bedrooms', y='price', color="blue", label='Bedrooms', data=pnumcol_clean)
sns.scatterplot(x='maximum_nights', y='price', color="red", label='Maximum Nights', data=pnumcol_clean)
sns.scatterplot(x='review_scores_rating', y='price', color="yellow", label='Review Scores', data=pnumcol_clean)
sns.scatterplot(x='bathrooms', y='price', color="orchid", label='Bathrooms', data=pnumcol_clean)
sns.scatterplot(x='accommodates', y='price', color="gray", label='Accommodates', data=pnumcol_clean)

# Mostrar la leyenda para identificar las series
plt.legend()

In [ ]:
# Graficar múltiples scatterplots en una sola gráfica
sns.scatterplot(x='bedrooms', y='price', color="blue", label='Bedrooms', data=pnumcol_clean)
plt.show()
sns.scatterplot(x='maximum_nights', y='price', color="red", label='Maximum Nights', data=pnumcol_clean)
plt.show()
sns.scatterplot(x='review_scores_rating', y='price', color="yellow", label='Review Scores', data=pnumcol_clean)
plt.show()
sns.scatterplot(x='bathrooms', y='price', color="orchid", label='Bathrooms', data=pnumcol_clean)
plt.show()
sns.scatterplot(x='accommodates', y='price', color="gray", label='Accommodates', data=pnumcol_clean)
plt.show()

In [ ]:
Vars_Indepmp= pnumcol_clean[['bedrooms','maximum_nights','review_scores_rating','bathrooms','accommodates']]
Var_Depmp= pnumcol_clean['price']

In [ ]:
modelmp= LinearRegression()

In [ ]:
#Verificamos la función relacionada al modelo
type(modelmp)

In [ ]:
#Ajustamos el modelo con las variables antes declaradas
modelmp.fit(X=Vars_Indepmp, y=Var_Depmp)

In [ ]:
#Verificamos los coeficientes obtenidos para el modelo ajustado
modelmp.__dict__

In [ ]:
#Evaluamos la eficiencia del modelo obtenido por medio del coeficiente R Determinación
modelmp.score(Vars_Indepmp,Var_Depmp)

In [ ]:
y_predmp= modelmp.predict(X=pnumcol_clean[['bedrooms','maximum_nights','review_scores_rating','bathrooms','accommodates']])
y_predmp

In [ ]:
pnumcol_clean.insert(0, 'Predicciones', y_predmp)

In [ ]:
# Crear un gráfico de dispersión para cada variable predictora
for column in ['bedrooms', 'maximum_nights', 'review_scores_rating', 'bathrooms', 'accommodates']:
    plt.figure(figsize=(8, 6))
    
    # Gráfico de los valores reales
    sns.scatterplot(x=column, y='price', data=pnumcol_clean, color="blue", label='Precio Real')
    
    # Gráfico de las predicciones
    sns.scatterplot(x=column, y='Predicciones', data=pnumcol_clean, color="red", label='Precio Predicho')
    
    # Añadir título y leyenda
    plt.title(f'Comparación entre {column} y Precio')
    plt.legend()
    
    # Mostrar la gráfica
    plt.show()

In [ ]:
#Corroboramos cual es el coeficiente de Determinación de nuestro modelo
coef_Determp=modelmp.score(X=Vars_Indepmp, y=Var_Depmp)
coef_Determp

In [ ]:
#Corroboramos cual es el coeficiente de Correlación de nuestro modelo
coef_Correlmp=np.sqrt(coef_Determp)
coef_Correlmp

In [ ]:
# Unimos los otros data frame para ver que esten limpios

dfp_limpiop = pd.concat([pnumcol_clean,numcolp,catcolp], axis=1)
dfp_limpiop.head(3)

***Convertimos a CSV los datos limpios***

In [ ]:
dfp_limpio.to_csv("Porto_limpio.csv")

In [ ]:
dfa_limpio.to_csv("Amsterdam Limpio.csv")